In [1]:
# リストを使って数理モデルを置き換える

In [2]:
import pulp

In [3]:
model=pulp.LpProblem()
Var=[pulp.LpVariable(f'Var{i}',cat=pulp.LpBinary)
     for i in range(3)]
model += (pulp.lpDot([1,2,3],Var)==2) # 内積の和
model += (pulp.lpSum(Var) ==1) # リストの要素の和
model.solve()
for v in Var:
    print(v.name, pulp.value(v))
print('Number', pulp.value(pulp.lpDot([1,2,3],Var)))

Var0 0.0
Var1 1.0
Var2 0.0
Number 2.0
